### Kids First: Genomic Analysis of Congenital Heart Defects and Acute Lymphoblastic Leukemia in Children


In [1]:
import os
import sys
import json
import requests
import pandas as pd 
import numpy as np
#import seaborn as sns
from pathlib import Path
from datetime import datetime
import pprint
#from lifelines import KaplanMeierFitter

pprint = pprint.PrettyPrinter(indent=1).pprint
# pprint() is now available to pretty-print any JSON

FHIR_SERVER = 'https://kf-api-fhir-service.kidsfirstdrc.org'

# Optional: Turn off SSL verification. Useful when dealing with a corporate proxy with self-signed certificates.
# This should be set to True unless you actually see certificate errors.
VERIFY_SSL = False

if not VERIFY_SSL:
    requests.packages.urllib3.disable_warnings()



# Kids First uses cookie-based authentication, so we've pre-fetched a cookie and made it available here.
#kf_cookie = requests.get("https://raw.githubusercontent.com/mitre/fhir-exercises/main/kf_cookie.txt", verify=VERIFY_SSL).text.rstrip()

full_cookie_path = os.path.expanduser('~/.keys/ncpi_prod_fhir_cookie.json')

with open(full_cookie_path) as f:
    cookies = json.load(f)
kf_cookie = cookies['Cookie']


# We make a requests.Session to ensure consistent headers/cookie across all the requests we make
s = requests.Session()
s.headers.update({'Accept': 'application/fhir+json'})
s.verify = VERIFY_SSL
s.cookies['AWSELBAuthSessionCookie-0'] = kf_cookie


# Test out the cookie by querying the server metadata
r = s.get(f"{FHIR_SERVER}/metadata")

if "<!DOCTYPE html>" in r.text:
    sys.stderr.write('ERROR: Could not authenticate with Kids First. The cookie may need to be updated')
    

# This helper method allows us to easily switch between printing an entire Bundle, or just the first 20 lines.
# Set truncate_for_github = False for actual use,
# or just replace the function with a `return bundle`

def print_bundle(bundle, truncate_for_github = False):
    if not truncate_for_github:
        return bundle
    
    lines = json.dumps(bundle, indent=2).split('\n')
    if len(lines) <= 20:
        return bundle
    else:   
        print('\n'.join(lines[:20]))
        print('...\nBundle truncated. Change the "print_bundle" function above to print the full content.')

In [2]:
# Resolves all pages for the bundle. Returns an array with all Bundles, including the original Bundle.
def resolve_pages(bundle):
    next_page_link = next(filter(lambda link: link['relation'] == 'next', bundle['link']), None)
    if next_page_link:
        next_page = s.get(next_page_link['url']).json()
        return [bundle] + resolve_pages(next_page)
    else:
        return [bundle]

# NOTE: No cell output.

def runQuery(query):
    r = s.get(f"{FHIR_SERVER}/{query}")
    first_bundle = r.json()
    all_bundles = resolve_pages(first_bundle)

    resources = [entry['resource'] for bundle in all_bundles for entry in bundle['entry']]
    print(f"Total  Resources: {len(resources)}")
    return resources

In [3]:
studies = runQuery(f"ResearchStudy")

studies

Total  Resources: 29


[{'resourceType': 'ResearchStudy',
  'id': '687202',
  'meta': {'versionId': '1',
   'lastUpdated': '2022-06-20T14:25:32.953+00:00',
   'source': '#RrhpFw6OupcziLQu',
   'profile': ['http://hl7.org/fhir/StructureDefinition/ResearchStudy']},
  'identifier': [{'use': 'official',
    'system': 'https://kf-api-dataservice.kidsfirstdrc.org/studies/',
    'value': 'SD_HGHFVPFD'},
   {'system': 'urn:kids-first:unique-string',
    'value': 'ResearchStudy-SD_HGHFVPFD'},
   {'system': 'https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=',
    'value': 'phs000424.v8.p2'}],
  'title': 'Common Fund (CF) Genotype-Tissue Expression Project (GTEx)',
  'status': 'completed',
  'category': [{'coding': [{'system': 'http://snomed.info/sct',
      'code': '86049000',
      'display': 'Malignant neoplasm, primary (morphologic abnormality)'}],
    'text': 'CANCER'}],
  'keyword': [{'coding': [{'code': 'Kids First'}]},
   {'coding': [{'code': 'KF-GTEX'}]}]},
 {'resourceType': 'ResearchStudy'

In [4]:
#phs = "phs002187.v1.p1"
#studies = runQuery(f"ResearchStudy?identifier=https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=|{phs}")
#study_id = studies[0]['id']
study_id = 48656
studies = runQuery(f"ResearchStudy?_id={study_id}")
print(study_id)
print(studies[0]['title'])
studies 

Total  Resources: 1
48656
Pediatric Brain Tumor Atlas: PNOC


[{'resourceType': 'ResearchStudy',
  'id': '48656',
  'meta': {'versionId': '2',
   'lastUpdated': '2022-01-18T22:57:56.745+00:00',
   'source': '#83Q7YI9F69RaW6Gn',
   'profile': ['http://hl7.org/fhir/StructureDefinition/ResearchStudy']},
  'identifier': [{'system': 'https://kf-api-dataservice.kidsfirstdrc.org/studies/',
    'value': 'SD_8Y99QZJJ'},
   {'system': 'urn:kids-first:unique-string',
    'value': 'ResearchStudy-SD_8Y99QZJJ'}],
  'title': 'Pediatric Brain Tumor Atlas: PNOC',
  'status': 'completed',
  'category': [{'coding': [{'system': 'http://snomed.info/sct',
      'code': '86049000',
      'display': 'Malignant neoplasm, primary (morphologic abnormality)'}],
    'text': 'CANCER'}],
  'keyword': [{'coding': [{'code': 'Pediatric Brain Tumor Atlas'}]},
   {'coding': [{'code': 'PBTA-PNOC'}]}]}]

In [5]:
patients = runQuery(f"Patient?_has:ResearchSubject:individual:study={study_id}")

Total  Resources: 84


In [6]:
import json
from collections import Counter
import pandas as pd

rTypes = Counter()

for p in patients:
    rTypes[p['resourceType']] += 1
    extCounter = Counter()
    if 'extension' in p:
        for e in p['extension']:
            extCounter[e['url']] +=1
    #print(extCounter)
print (json.dumps(rTypes, indent=3))

{
   "Patient": 84
}


In [7]:
n=53
patient_id = patients[n]['id']
patients[n]


{'resourceType': 'Patient',
 'id': '48592',
 'meta': {'versionId': '2',
  'lastUpdated': '2021-11-16T08:30:46.741+00:00',
  'source': '#8OzwRRXz63pnmm2D',
  'profile': ['http://hl7.org/fhir/StructureDefinition/Patient'],
  'tag': [{'code': 'SD_8Y99QZJJ'}]},
 'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race',
   'extension': [{'url': 'text', 'valueString': 'White'},
    {'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
      'code': '2106-3',
      'display': 'White'}}]},
  {'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity',
   'extension': [{'url': 'text', 'valueString': 'Not Hispanic or Latino'},
    {'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
      'code': '2186-5',
      'display': 'Not Hispanic or Latino'}}]}],
 'identifier': [{'value': 'C3078075'},
  {'system': 'https://kf-api-dataservice.kidsfirstdrc.org/participants/',
   'value': 'PT_YR

We can see the data in more compact form via a DataFrame. This shows that the same set of three attributes are available for all patients.

In [8]:
patient_list = []
patient_ids = []
std_attributes = {'gender':'','birthDate':'','maritalStatus':'','multipleBirthBoolean':''}
for p in patients:
    rTypes[p['resourceType']] += 1
    extCounter = Counter()
    for e in p['extension']:
        ext = e['url'].split('/')[-1]
        extCounter[ext] += 1
    for att in std_attributes.keys():
        if att in p:
            extCounter[att] += 1
    patient_list.append(extCounter)
    patient_ids.append(p['id'])

pd.set_option("display.max_rows", None, "display.max_columns", None)
pdf = pd.DataFrame(patient_list)    
pdf

,us-core-race,us-core-ethnicity,gender
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
5,1,1,1
6,1,1,1
7,1,1,1
8,1,1,1
9,1,1,1


The attributes above are informative but are unlikely to part of a meaningful query in building an asthma cohort.

What else might we look at? Let's try Observations.

In [9]:
import pandas as pd

resources = runQuery(f"Patient?_has:ResearchSubject:individual:study={study_id}&_revinclude=Observation:subject")

observations = []
obsCounter  = Counter()
codeCounter = Counter()
vccCounter = Counter()
printObsCounts = False
for r in resources:

    if r['resourceType'] == 'Observation':
        obsCounter[r['subject']['reference']] +=1
        obs_display_name = r['code']['coding'][0]['display']
        vcc_text = r['valueCodeableConcept']['text']
        codeCounter[obs_display_name] +=1
        vccCounter[vcc_text] +=1
        observations.append(r)



#Summarize
print(f"Number of patients with observations {len(obsCounter.keys())}")

if printObsCounts:
    print("Observation count per patient")
    print(json.dumps(obsCounter, indent=3))
print("Coding counts")
#print(json.dumps(codeCounter, indent=3))
df = pd.DataFrame.from_dict(codeCounter,  orient='index')
pd.set_option("display.max_rows", None, "display.max_columns", None)
display(df)
vccdf= pd.DataFrame.from_dict(vccCounter,  orient='index')
display(vccdf)

Total  Resources: 162
Number of patients with observations 78
Coding counts


,0
Clinical status,78


,0
Deceased,37
Alive,41


In [10]:
observations[1]

{'resourceType': 'Observation',
 'id': '48835',
 'meta': {'versionId': '2',
  'lastUpdated': '2021-11-16T08:30:55.161+00:00',
  'source': '#oDhSRqN6No2PCaln',
  'profile': ['https://nih-ncpi.github.io/ncpi-fhir-ig/StructureDefinition/vital-status'],
  'tag': [{'code': 'SD_8Y99QZJJ'}]},
 'identifier': [{'system': 'https://kf-api-dataservice.kidsfirstdrc.org/outcomes/',
   'value': 'OC_17WZQWAC'},
  {'system': 'urn:kids-first:unique-string',
   'value': 'Observation-SD_8Y99QZJJ-OC_17WZQWAC'}],
 'status': 'final',
 'code': {'coding': [{'system': 'http://snomed.info/sct',
    'code': '263493007',
    'display': 'Clinical status'}],
  'text': 'Clinical status'},
 'subject': {'reference': 'Patient/48621'},
 'valueCodeableConcept': {'coding': [{'system': 'http://snomed.info/sct',
    'code': '438949009',
    'display': 'Alive'}],
  'text': 'Alive'}}

In [11]:
print(f"Patient/{patient_id}")
documents = runQuery(f"DocumentReference?subject=Patient/{patient_id}")



Patient/48592
Total  Resources: 61


In [41]:
for d in documents:
    print(json.dumps(d['content'][0]['attachment'], indent=3))
    print('_'*40)
    print(json.dumps(d['content'][1], indent=3))
    print('_'*80)

{
   "url": "drs://data.kidsfirstdrc.org/ee38b0fe-8de8-47f5-9502-96deacfe09d2"
}
________________________________________
{
   "attachment": {
      "extension": [
         {
            "url": "https://nih-ncpi.github.io/ncpi-fhir-ig/StructureDefinition/file-size",
            "valueDecimal": 10311
         },
         {
            "url": "https://nih-ncpi.github.io/ncpi-fhir-ig/StructureDefinition/hashes",
            "valueCodeableConcept": {
               "coding": [
                  {
                     "display": "md5"
                  }
               ],
               "text": "15d2667aa2df5eeb8edb9e8017e7b833"
            }
         },
         {
            "url": "https://nih-ncpi.github.io/ncpi-fhir-ig/StructureDefinition/hashes",
            "valueCodeableConcept": {
               "coding": [
                  {
                     "display": "sha256"
                  }
               ],
               "text": "9969b80770b9fd85b48e4ecb4296742945969ae228601d391a06bf

{
   "attachment": {
      "extension": [
         {
            "url": "https://nih-ncpi.github.io/ncpi-fhir-ig/StructureDefinition/file-size",
            "valueDecimal": 87637
         },
         {
            "url": "https://nih-ncpi.github.io/ncpi-fhir-ig/StructureDefinition/hashes",
            "valueCodeableConcept": {
               "coding": [
                  {
                     "display": "md5"
                  }
               ],
               "text": "ae1a9a81fbae04b4cb10c78e0c5b6f4b"
            }
         },
         {
            "url": "https://nih-ncpi.github.io/ncpi-fhir-ig/StructureDefinition/hashes",
            "valueCodeableConcept": {
               "coding": [
                  {
                     "display": "sha256"
                  }
               ],
               "text": "a559eb305018456b5fdd261ce6feb1583d50b5ce90b6b401af3d6c18287a102a"
            }
         }
      ],
      "url": "s3://kf-study-us-east-1-prd-sd-8y99qzjj/harmonized-data/simple

In [13]:
def download(url, file_path):
    '''Download a file from a URL to a local file path'''
    with open(os.path.expanduser(file_path), "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [15]:
exp_docs = runQuery("DocumentReference?type:text=Gene%20Expression&security-label=U&_tag=SD_8Y99QZJJ")

Total  Resources: 64


In [17]:
from fasp.loc import kfDRSClient

cl = kfDRSClient("~/.keys/kf_credentials.json")

for d in exp_docs:
    url = d['content'][0]['attachment']['url']
    print(d['type']['text'])

    print(url)
    drs_id = url.split('/')[-1]
    drs_response = cl.get_object(drs_id)
    #print(json.dumps(drs_response, indent=3))
    file_name = drs_response["name"]
    print(file_name)
    d_url= cl.get_access_url(drs_id, 's3')
    download(d_url,f'/yourpath/{file_name}')



    print(d['content'][1]['format']['display'])

    print('_'*80)

Gene Expression
drs://data.kidsfirstdrc.org/be883ef8-4049-415f-8444-3072fae1c71c
487ba87c-053d-47dc-a5bc-2e7e77733ace.rsem.genes.results.gz
tsv
________________________________________________________________________________
Gene Expression
drs://data.kidsfirstdrc.org/b841fc14-0769-4a95-9824-4d70b75cd5d6
808baf95-9a60-4ecc-a393-5910d054b581.rsem.genes.results.gz
tsv
________________________________________________________________________________
Gene Expression
drs://data.kidsfirstdrc.org/bff411a0-b356-4042-b3eb-283556935472
9ece8e3a-6e4d-4fb4-b12f-c5ab2000fa38.rsem.genes.results.gz
tsv
________________________________________________________________________________
Gene Expression
drs://data.kidsfirstdrc.org/65ee3d14-a471-42f2-857f-e75dbd5e8756
fe12f0da-e170-4345-b55c-889c27333ab9.rsem.genes.results.gz
tsv
________________________________________________________________________________
Gene Expression
drs://data.kidsfirstdrc.org/8b8f6e3b-65be-4f93-a259-75fc306ba14a
9ece8e3a-6e4d-4fb4-

tsv
________________________________________________________________________________
Gene Expression
drs://data.kidsfirstdrc.org/2fab40b4-4947-4fe0-ad26-5c43b02d86ca
ab2e7fcf-a4d2-46f0-8235-fde6708d2cc1.kallisto.abundance.tsv.gz
tsv
________________________________________________________________________________
Gene Expression
drs://data.kidsfirstdrc.org/878676f7-bff2-4613-816d-ec5a65a8920f
1c362725-01bd-4590-b054-c7ad04e5d6cb.rsem.genes.results.gz
tsv
________________________________________________________________________________
Gene Expression
drs://data.kidsfirstdrc.org/561da2bf-4fda-4b31-ba9b-408c881c8572
1c7dd81c-f264-4cc8-bba4-373ba5bf89f9.kallisto.abundance.tsv.gz
tsv
________________________________________________________________________________
Gene Expression
drs://data.kidsfirstdrc.org/ee44fe9a-4370-47c4-8526-d6ed83a33d60
e3e8db5f-e846-4ceb-8ac2-1cf62c00577a.kallisto.abundance.tsv.gz
tsv
________________________________________________________________________________
Gen

In [21]:
import pandas as pd

def create_dataframe(documents):
    records = []
    patient_counter = Counter()
    for d in documents:
        url = d['content'][0]['attachment']['url']
        type = d['type']['text']
        drs_id = url.split('/')[-1]
        drs_response = cl.get_object(drs_id)
        #print(json.dumps(drs_response, indent=3))
        file_name = drs_response["name"]
        #print(file_name)
        subject_id = d['subject']['reference']
        patient_counter[subject_id]+=1
        format = d['content'][1]['format']['display']
        records.append({"subject_id":subject_id,"file_name":file_name,
                        "format":format,"type":type,"drs_id":drs_id})

    print('_'*80)
    #print(json.dumps(patient_counter, indent=3))
    df = pd.DataFrame(records)
    return df

In [23]:
df = create_dataframe(exp_docs)
df

________________________________________________________________________________


,subject_id,file_name,format,type,drs_id
0,Patient/48604,487ba87c-053d-47dc-a5bc-2e7e77733ace.rsem.gene...,tsv,Gene Expression,be883ef8-4049-415f-8444-3072fae1c71c
1,Patient/48598,808baf95-9a60-4ecc-a393-5910d054b581.rsem.gene...,tsv,Gene Expression,b841fc14-0769-4a95-9824-4d70b75cd5d6
2,Patient/48577,9ece8e3a-6e4d-4fb4-b12f-c5ab2000fa38.rsem.gene...,tsv,Gene Expression,bff411a0-b356-4042-b3eb-283556935472
3,Patient/48576,fe12f0da-e170-4345-b55c-889c27333ab9.rsem.gene...,tsv,Gene Expression,65ee3d14-a471-42f2-857f-e75dbd5e8756
4,Patient/48577,9ece8e3a-6e4d-4fb4-b12f-c5ab2000fa38.kallisto....,tsv,Gene Expression,8b8f6e3b-65be-4f93-a259-75fc306ba14a
5,Patient/48595,0524d200-76d3-4929-87d1-d68784df44f1.rsem.gene...,tsv,Gene Expression,eb3d5066-5af5-4935-82aa-13fe9f68c3d5
6,Patient/48576,fe12f0da-e170-4345-b55c-889c27333ab9.kallisto....,tsv,Gene Expression,d4541867-d8f1-45a0-b889-4229b53e1ab9
7,Patient/48594,1a39a1bf-e744-42a9-ad5f-d89d28bb5fa5.rsem.gene...,tsv,Gene Expression,fdfc1715-a963-4914-9bfc-ae5e3c1e6dca
8,Patient/48594,1a39a1bf-e744-42a9-ad5f-d89d28bb5fa5.kallisto....,tsv,Gene Expression,8d5fd864-8345-4891-a152-014ef7864250
9,Patient/48610,7197834a-4d9d-4c94-beb7-d1fe20514aca.rsem.gene...,tsv,Gene Expression,21adfc3b-a5ac-495b-9fe7-945038c12d70


In [71]:
folder = '/Users/forei/Downloads/study1'

def download_study_files(folder):
    for d in exp_docs:
        url = d['content'][0]['attachment']['url']
        print(d['type']['text'])

        print(url)
        drs_id = url.split('/')[-1]
        drs_response = cl.getObject(drs_id)
        #print(json.dumps(drs_response, indent=3))
        file_name = drs_response["name"]
        print(file_name)
        d_url= cl.getAccessURL(drs_id, 's3')
        download(d_url,f'{folder}/{file_name}')



    print(d['content'][1]['format']['display'])

    print('_'*80)

In [58]:
def check_access(documents, verbose=True):
    #access_id_counter = Counter()
    accessible_counter = Counter()
    inaccessible_counter = Counter()
    patient_counter = Counter()
    for d in documents:
        url = d['content'][0]['attachment']['url']
        drs_id = url.split('/')[-1]
        drs_response = cl.getObject(drs_id)
        subject_id = d['subject']['reference']
        #print(subject_id)
        patient_counter[subject_id]+=1
        #for a in drs_response['access_methods']:
            #access_id_counter[a['access_id']] +=1

        acc_url = cl.getAccessURL(drs_id, 's3')
        file_format = d['content'][1]['format']['display']
        if acc_url != None:
            print(acc_url)
            accessible_counter[file_format] +=1
            if verbose:
                    print(d['type']['text'])
                    
                    print(file_format)
        else:
            inaccessible_counter[file_format] +=1
        #print(d['content'][1]['format']['display'])
    #print(access_id_counter)
    print("Accessible files")
    print(accessible_counter)
    print("Inaccessible files")
    print(inaccessible_counter)
    print(patient_counter)




In [48]:
check_access(documents, verbose=True)

Unauthorized for that DRS id
Unauthorized for that DRS id
Variant Calls Index
tbi
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Not Reported
maf
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Not Reported
maf
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Variant Calls
vcf
Variant Calls
vcf
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unauthorized for that DRS id
Unautho

Total  Resources: 64


________________________________________________________________________________
{
   "Patient/48577": 2,
   "Patient/48576": 2,
   "Patient/48595": 2,
   "Patient/48598": 2,
   "Patient/48594": 2,
   "Patient/48599": 2,
   "Patient/48604": 2,
   "Patient/48610": 2,
   "Patient/48614": 2,
   "Patient/48605": 2,
   "Patient/48611": 2,
   "Patient/48606": 4,
   "Patient/48613": 2,
   "Patient/48627": 2,
   "Patient/48615": 2,
   "Patient/48634": 4,
   "Patient/48619": 2,
   "Patient/48612": 2,
   "Patient/48628": 2,
   "Patient/48622": 2,
   "Patient/48630": 2,
   "Patient/48620": 2,
   "Patient/48645": 2,
   "Patient/48639": 2,
   "Patient/48648": 2,
   "Patient/48642": 2,
   "Patient/48653": 2,
   "Patient/48655": 2,
   "Patient/48654": 2,
   "Patient/48652": 2
}


In [97]:
df

,subject_id,file_name,format,type,drs_id
0,Patient/48577,9ece8e3a-6e4d-4fb4-b12f-c5ab2000fa38.kallisto....,tsv,Gene Expression,8b8f6e3b-65be-4f93-a259-75fc306ba14a
1,Patient/48576,fe12f0da-e170-4345-b55c-889c27333ab9.rsem.gene...,tsv,Gene Expression,65ee3d14-a471-42f2-857f-e75dbd5e8756
2,Patient/48595,0524d200-76d3-4929-87d1-d68784df44f1.rsem.gene...,tsv,Gene Expression,eb3d5066-5af5-4935-82aa-13fe9f68c3d5
3,Patient/48577,9ece8e3a-6e4d-4fb4-b12f-c5ab2000fa38.rsem.gene...,tsv,Gene Expression,bff411a0-b356-4042-b3eb-283556935472
4,Patient/48598,808baf95-9a60-4ecc-a393-5910d054b581.rsem.gene...,tsv,Gene Expression,b841fc14-0769-4a95-9824-4d70b75cd5d6
5,Patient/48594,1a39a1bf-e744-42a9-ad5f-d89d28bb5fa5.rsem.gene...,tsv,Gene Expression,fdfc1715-a963-4914-9bfc-ae5e3c1e6dca
6,Patient/48576,fe12f0da-e170-4345-b55c-889c27333ab9.kallisto....,tsv,Gene Expression,d4541867-d8f1-45a0-b889-4229b53e1ab9
7,Patient/48599,2607779b-c42c-4a2d-95fb-70edcf73517a.rsem.gene...,tsv,Gene Expression,3f72c451-01c5-4f61-9e38-3dd5c9684369
8,Patient/48604,487ba87c-053d-47dc-a5bc-2e7e77733ace.rsem.gene...,tsv,Gene Expression,be883ef8-4049-415f-8444-3072fae1c71c
9,Patient/48594,1a39a1bf-e744-42a9-ad5f-d89d28bb5fa5.kallisto....,tsv,Gene Expression,8d5fd864-8345-4891-a152-014ef7864250


In [ ]:
exp_docs2 = runQuery("DocumentReference?type:text=Gene%20Expression&security-label=U&_tag=SD_8Y99QZJJ")